# Data preparation/Chunking

In [ ]:
!ls data

legal.json          newsfaq.json        newsguidelines.json security.json


In [ ]:
import json
import os
import spacy
import faiss
from sentence_transformers import SentenceTransformer
import openai
from dotenv import load_dotenv
import re

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
nlp = spacy.load("en_core_web_sm")

def process_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
        content = data["content"]
        url = data["url"]
        doc = nlp(content)

        return [{"text": sent.text, "url": url} for sent in doc.sents]

chunks = [
    chunk
    for file in os.listdir("data")
    for chunk in process_file(os.path.join("data", file))
]

chunks = [{"id": i, **chunk} for i, chunk in enumerate(chunks)]

with open("chunks.json", "w") as f:
    json.dump(chunks, f)

# Vector store

In [ ]:
sentences = [chunk["text"] for chunk in chunks]

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

embeddings = model.encode(sentences, show_progress_bar=True)

In [ ]:
faiss_index = faiss.IndexFlatIP(model.get_sentence_embedding_dimension())
faiss_index.add(embeddings)

# Retrieval/Prompt preparation

In [ ]:
base_prompt = """You are an AI assistant. Your task is to understand the user question, and provide an answer using the provided contexts. Every answer you generate should have citations in this pattern  "Answer [position].", for example: "Earth is round [1][2].," if it's relevant.

Your answers are correct, high-quality, and written by an domain expert. If the provided context does not contain the answer, simply state, "The provided contexts does not have the answer."

User question: {}

Contexts:
{}
"""

In [ ]:
k = 50
question = "Why HackerNews is so popular?"

query_embedding = model.encode([question])
distances, indices = faiss_index.search(query_embedding, k)

context = "\n".join([f"{i}. {sentences[index]}" for i, index in enumerate(indices[0])])
prompt = f"{base_prompt.format(question, context)}"

# Answer Generation

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k-0613",
    temperature=0,
    messages=[
        {"role": "system", "content": prompt},
    ]
)

print(response.choices[0].message.content)